# ¿Cuánto deberían valer las propiedades en Milwaukee, Wisconsin? (Parte II)

## Objetivos

En los últimos casos, hemos visto varias técnicas utilizadas para diagnosticar la aplicabilidad de los modelos de regresión lineal.

## Introducción 

**Problema empresarial.** Su tarea es **crear un modelo para predecir los precios de las propiedades en la ciudad de Milwaukee, Wisconsin**.

**Contexto analítico.**  el conjunto de datos consta de ventas de propiedades (comerciales y residenciales) en Milwaukee, Wisconsin, de 2002 a 2018. 


# Preparar data

In [1]:
### Load relevant packages
import pandas                  as pd
import numpy                   as np
import matplotlib.pyplot       as plt
import seaborn                 as sns
import statsmodels.formula.api as smf
import statsmodels.api         as sm
import scipy

%matplotlib inline
plt.style.use('ggplot')
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [2]:
data = pd.read_csv("2002-2018-property-sales-data.csv",
    dtype = { # Categorias
        "PropType": "category",
        "District": "category",
        "Extwall": "category",
        "Nbhd": "category",
        "Style": "category",
    },
    parse_dates=["Sale_date"],
)
def remove_unused_categories(data):
    """ The `remove_unused_categories` method in pandas
        removes categories from a Series if there are no
        elements of that category.
        
        This function is a convenience function that removes
        unused categories for all categorical columns
        of a data frame.
        
        The reason this is useful is that when we
        fit a linear regression, `statsmodels` will
        create a coefficient for every category in a column,
        and so unused categories pollute the results.
    """
    for cname in data:
        col = data[cname]
        if pd.api.types.is_categorical_dtype(col):
            data[cname] = col.cat.remove_unused_categories()
    return data

clean = np.where(
    (data["Sale_price"] > 2000) & # Importante
    (data["Year_Built"] > 1800) &
    (data["Fin_sqft"] > 0) & # Condicion logica
    (data["Lotsize"] > 0)  & # Condicion Logica
    (data["PropType"] == "Residential")
    )
data_clean = data.iloc[clean].copy()
remove_unused_categories(data_clean).head()

/var/folders/wt/dd5nh3v12zvd92jwl2f9nt400000gn/T/ipykernel_79723/1550737232.py:27: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(col):


,PropType,Taxkey,Address,CondoProject,District,Nbhd,Style,Extwall,Stories,Year_Built,Nr_of_rms,Fin_sqft,Units,Bdrms,Fbath,Hbath,Lotsize,Sale_date,Sale_price
10,Residential,3080013000,3033 N 35TH ST,NaN,7,2960,AP 1,Frame,2.0,1913,0,3476,4,9,1,0,5040,2002-02-01,42000
51,Residential,3190434000,1908 E WEBSTER PL,NaN,3,3170,Rm or Rooming House,Frame,2.0,1897,0,1992,4,2,2,0,2880,2002-05-01,145000
67,Residential,3891722000,812 N 25TH ST,NaN,4,3040,Rm or Rooming House,Frame,2.0,1907,0,2339,6,0,1,0,3185,2002-06-01,30000
116,Residential,3880628000,959 N 34TH ST,NaN,4,2300,AP 1,Frame,2.0,1890,0,2329,4,4,1,0,5781,2002-10-01,66500
134,Residential,3880406000,3209 W WELLS ST,NaN,4,2300,Mansion,Stone,2.5,1891,0,7450,2,7,6,0,15600,2002-11-01,150500


Debemos dividir aleatoriamente los datos en un conjunto de entrenamiento y un conjunto de prueba. El **conjunto de entrenamiento** es aquel en el que entrenamos y ajustamos nuestro modelo de regresión lineal múltiple. Luego ejecutamos nuestro modelo ajustado en el **conjunto de prueba** y comparamos sus predicciones con los datos de la variable de respuesta del conjunto de prueba real para evaluar su rendimiento.

In [4]:
#### RESPUESTA AQUI
np.random.seed(135568109) 
ndata = len(data_clean)
idx_train = np.random.choice(range(ndata),int(0.8*ndata),replace=False)
idx_test  = np.asarray(list(set(range(ndata)) - set(idx_train)))
train     = data_clean.iloc[idx_train] # train
test      = data_clean.iloc[idx_test]  # test
print(train.shape) # 19,312 
print(test.shape)  #  4,829 

(19556, 19)
(4889, 19)


Ahora apodemos hacer un modelo para poder predecir `Sales` en funcion de `District + Units+Fin_sqft`

Analiza los resultados , hacen sentido?

In [5]:
#### RESPUESTA AQUI
model_log = smf.ols(formula = "Sale_price ~ District + Units"
                           "+ Fin_sqft", 
                 data = train).fit()
model_log.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Sale_price   R-squared:                       0.674
Model:                            OLS   Adj. R-squared:                  0.674
Method:                 Least Squares   F-statistic:                     2522.
Date:                Thu, 30 Jan 2025   Prob (F-statistic):               0.00
Time:                        18:22:52   Log-Likelihood:            -2.3898e+05
No. Observations:               19556   AIC:                         4.780e+05
Df Residuals:                   19539   BIC:                         4.781e+05
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept       2.111e+04   2037.224     10.361      0.000    1.71e+04    2.51e+04
District[T.10]  4.993e+04   2053.420     24.318      0.000    4.59e+04     5.4e+04
District[T.11]  7.286e+04   1974.069     36.908      0.000     6.9e+04    7.67e+04
District[T.12] -3327.9414   3134.914     -1.062      0.288   -9472.640    2816.758
District[T.13]  7.162e+04   2044.727     35.025      0.000    6.76e+04    7.56e+04
District[T.14]  8.781e+04   2037.479     43.097      0.000    8.38e+04    9.18e+04
District[T.15] -4.997e+04   3003.689    -16.638      0.000   -5.59e+04   -4.41e+04
District[T.2]   1.867e+04   2275.581      8.206      0.000    1.42e+04    2.31e+04
District[T.3]   1.401e+05   2336.647     59.938      0.000    1.35e+05    1.45e+05
District[T.4]  -6.818e+04   4719.596    -14.446      0.000   -7.74e+04   -5.89e+04
District[T.5]   5.548e+04   1973.580     28.113      0.000    5.16e+04    5.94e+04
District[T.6]  -5797.9472   2753.277     -2.106      0.035   -1.12e+04    -401.289
District[T.7]  -1.421e+04   2442.523     -5.817      0.000    -1.9e+04   -9420.894
District[T.8]  -1088.2739   2637.765     -0.413      0.680   -6258.519    4081.971
District[T.9]   4.206e+04   2385.842     17.629      0.000    3.74e+04    4.67e+04
Units          -6.999e+04    943.776    -74.160      0.000   -7.18e+04   -6.81e+04
Fin_sqft          99.1541      0.723    137.224      0.000      97.738     100.570
==============================================================================
Omnibus:                    14105.557   Durbin-Watson:                   2.003
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1397399.716
Skew:                           2.710   Prob(JB):                         0.00
Kurtosis:                      44.056   Cond. No.                     3.31e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.31e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Ahora intentemos realizar algunas transformaciones para entender si se mejoran nuestros resultados
Puedes usar esto como referencia para hacer el modelo 
```python
model_log = smf.ols(formula = "np.log(Sale_price) ~ District + Units"
                           "+ np.log(Fin_sqft)", 
                 data = train).fit()
```

In [6]:
#### RESPUESTA AQUI
model_log = smf.ols(formula = "np.log(Sale_price) ~ District + Units"
                           "+ np.log(Fin_sqft)", 
                 data = train).fit()
model_log.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     np.log(Sale_price)   R-squared:                       0.607
Model:                            OLS   Adj. R-squared:                  0.607
Method:                 Least Squares   F-statistic:                     1886.
Date:                Thu, 30 Jan 2025   Prob (F-statistic):               0.00
Time:                        18:23:03   Log-Likelihood:                -7612.5
No. Observations:               19556   AIC:                         1.526e+04
Df Residuals:                   19539   BIC:                         1.539e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            5.2685      0.063     84.177      0.000       5.146       5.391
District[T.10]       0.6211      0.015     41.545      0.000       0.592       0.650
District[T.11]       0.8147      0.014     56.736      0.000       0.787       0.843
District[T.12]      -0.0125      0.023     -0.546      0.585      -0.057       0.032
District[T.13]       0.7995      0.015     53.747      0.000       0.770       0.829
District[T.14]       0.8624      0.015     58.160      0.000       0.833       0.891
District[T.15]      -0.4609      0.022    -21.077      0.000      -0.504      -0.418
District[T.2]        0.3029      0.017     18.298      0.000       0.270       0.335
District[T.3]        1.0236      0.017     60.776      0.000       0.991       1.057
District[T.4]       -0.2284      0.034     -6.670      0.000      -0.296      -0.161
District[T.5]        0.6830      0.014     47.556      0.000       0.655       0.711
District[T.6]       -0.1563      0.020     -7.799      0.000      -0.196      -0.117
District[T.7]       -0.0693      0.018     -3.896      0.000      -0.104      -0.034
District[T.8]        0.1562      0.019      8.136      0.000       0.119       0.194
District[T.9]        0.5684      0.017     32.742      0.000       0.534       0.602
Units               -0.3476      0.007    -50.168      0.000      -0.361      -0.334
np.log(Fin_sqft)     0.8642      0.009     94.900      0.000       0.846       0.882
==============================================================================
Omnibus:                     4417.978   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            26736.926
Skew:                          -0.949   Prob(JB):                         0.00
Kurtosis:                       8.405   Cond. No.                         186.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Variables categoricas

Quizás haya notado que hay más de una docena de coeficientes para `Distrito` arriba. Esto se debe a que `Distrito` es una variable categórica y, para las características categóricas, se obtiene un coeficiente para todas las categorías excepto una. Si solo hay dos categorías (por ejemplo, género), esto es intuitivo: la característica se convierte en una columna de ceros y unos antes de introducirla en la regresión, donde se trata como una variable numérica regular.

Cuando hay más de dos categorías, una categoría se designa como la categoría de "referencia" o "línea de base", y se crean columnas "ficticias" de unos y ceros para todas las demás categorías. Tomemos un ejemplo ficticio con tres categorías y cinco filas:

| Categoría |
|--------------|
| A |
| B |
| C |
| C |
| A |

Antes de ajustar la regresión lineal, la columna `Categoría` se transforma en **dos** columnas “ficticias” (no se agrega una columna para la categoría de referencia).

La primera columna es 1 si el distrito es `B` y 0 en caso contrario, mientras que la segunda columna es 1 si el distrito es `C` y 0 en caso contrario. Obtenemos:

| Categoría_B | Categoría_C |
|--------------|-------------|
| 0 | 0 |
| 1 | 0 |
| 0 | 1 |
| 0 | 1 |
| 0 | 0 |

Las columnas ficticias se introducen en la regresión lineal y se tratan como variables numéricas regulares. Esta técnica se denomina **codificación one-hot** y se puede realizar manualmente con la función pandas `pd.get_dummies()`:

In [3]:
df_dummy=pd.DataFrame({'Category':['A','B','C','C','A']})
pd.get_dummies(df_dummy,columns=['Category'], drop_first=True) 

,Category_B,Category_C
0,False,False
1,True,False
2,False,True
3,False,True
4,False,False


Al igual que en el caso binario, la elección de la línea de base cambia los coeficientes y su interpretación; el coeficiente `Distrito[T.3]` de 1.0236 debe interpretarse como la diferencia en los resultados previstos entre los distritos 3 y 1. Pero la elección de la línea de base no afecta las predicciones ni el rendimiento del modelo, por lo que la mayoría del software elegirá arbitrariamente una categoría como línea de base sin previo aviso (a menudo la primera alfabéticamente).

Ahora hay un nuevo desafio 

Agrega la columna `Style` a nuestro último modelo de regresión. Hazlo de dos maneras: primero transformando la variable `Style` usando `pd.get_dummies()` y luego sin la transformación. Verifica que obtengas los mismos resultados. De acuerdo con este modelo, ¿qué propiedad style es la más deseable?

**Sugerencia:** La variable `Style` contiene nombres, como `Residence O/S`, que pueden ser problemáticos al escribir fórmulas `smf.ols` (consulta la [Sintaxis de Pasty](https://patsy.readthedocs.io/en/latest/index.html)). Puedes usar la sintaxis `Pasty` entre comillas `Q()` en los nombres de variables para evitar este problema:
```python
formula="np.log(Sale_price) ~Q("Residence O/S")+...
```

In [8]:
#### RESPUESTA AQUI
# Ordenar Style
Styles=list(train['Style'].unique())
Styles.sort() #Just to keep track of the baseline category.

train_dummy=pd.get_dummies(train,columns=['Style'],prefix='',prefix_sep='',drop_first=True)

# Formula
formula0="np.log(Sale_price) ~ "
for style in Styles[1:]:
    formula0+='Q("'+style+'")+'
formula0=formula0+'District + Units+ np.log(Fin_sqft)'
print(formula0)
# Modelo
model_style_1 = smf.ols(formula = formula0, 
                 data = train_dummy).fit()
model_style_1.summary()

np.log(Sale_price) ~ Q("Bi-Level")+Q("Cape Cod")+Q("Colonial")+Q("Cottage")+Q("Dplx Bungalow")+Q("Duplex N/S")+Q("Duplex O/S")+Q("Duplex-Cottage")+Q("Mansion")+Q("Milwaukee Bungalow")+Q("Ranch")+Q("Residence O/S")+Q("Rm or Rooming House")+Q("Split Level")+Q("Townhouse")+Q("Triplex")+Q("Tudor")+District + Units+ np.log(Fin_sqft)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     np.log(Sale_price)   R-squared:                       0.636
Model:                            OLS   Adj. R-squared:                  0.635
Method:                 Least Squares   F-statistic:                     1034.
Date:                Thu, 30 Jan 2025   Prob (F-statistic):               0.00
Time:                        18:23:54   Log-Likelihood:                -6859.4
No. Observations:               19556   AIC:                         1.379e+04
Df Residuals:                   19522   BIC:                         1.405e+04
Df Model:                          33                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                            4.4606      0.165     27.093      0.000       4.138       4.783
Q("Bi-Level")[T.True]                0.1548      0.130      1.195      0.232      -0.099       0.409
Q("Cape Cod")[T.True]                0.2740      0.121      2.260      0.024       0.036       0.512
Q("Colonial")[T.True]                0.3373      0.121      2.778      0.005       0.099       0.575
Q("Cottage")[T.True]                -0.0194      0.122     -0.159      0.874      -0.259       0.220
Q("Dplx Bungalow")[T.True]          -0.1922      0.101     -1.911      0.056      -0.389       0.005
Q("Duplex N/S")[T.True]             -0.0573      0.101     -0.568      0.570      -0.255       0.140
Q("Duplex O/S")[T.True]             -0.2727      0.100     -2.717      0.007      -0.469      -0.076
Q("Duplex-Cottage")[T.True]         -0.4176      0.105     -3.994      0.000      -0.623      -0.213
Q("Mansion")[T.True]                 0.3672      0.128      2.876      0.004       0.117       0.617
Q("Milwaukee Bungalow")[T.True]      0.1290      0.121      1.064      0.288      -0.109       0.367
Q("Ranch")[T.True]                   0.3379      0.121      2.786      0.005       0.100       0.576
Q("Residence O/S")[T.True]           0.0998      0.121      0.824      0.410      -0.138       0.337
Q("Rm or Rooming House")[T.True]    -0.2441      0.155     -1.577      0.115      -0.547       0.059
Q("Split Level")[T.True]             0.1984      0.125      1.586      0.113      -0.047       0.444
Q("Townhouse")[T.True]              -0.0811      0.104     -0.782      0.434      -0.284       0.122
Q("Triplex")[T.True]                -0.2118      0.089     -2.392      0.017      -0.385      -0.038
Q("Tudor")[T.True]                   0.3637      0.124      2.941      0.003       0.121       0.606
District[T.10]                       0.6678      0.015     45.862      0.000       0.639       0.696
District[T.11]                       0.7631      0.014     54.638      0.000       0.736       0.790
District[T.12]                       0.1575      0.023      6.765      0.000       0.112       0.203
District[T.13]                       0.7626      0.014     52.997      0.000       0.734       0.791
District[T.14]                       0.9436      0.015     64.681      0.000       0.915       0.972
District[T.15]                      -0.3656      0.021    -17.020      0.000      -0.408      -0.323
District[T.2]                        0.2498      0.016     15.489      0.000       0.218       0.281
District[T.3]                        1.1182      0.017     65.617      0.000       1.085       1.152
District[T.4]                       -0.1705      0.034     -5.076      0.000      -0.236      -0.105
District[T.5]                        0.6298      0.01

In [9]:
model_style = smf.ols(formula = "np.log(Sale_price) ~ Style + District + Units"
                           "+ np.log(Fin_sqft)", 
                 data = train).fit()
model_style.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     np.log(Sale_price)   R-squared:                       0.636
Model:                            OLS   Adj. R-squared:                  0.635
Method:                 Least Squares   F-statistic:                     1034.
Date:                Thu, 30 Jan 2025   Prob (F-statistic):               0.00
Time:                        18:24:18   Log-Likelihood:                -6859.4
No. Observations:               19556   AIC:                         1.379e+04
Df Residuals:                   19522   BIC:                         1.405e+04
Df Model:                          33                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        4.4606      0.165     27.093      0.000       4.138       4.783
Style[T.Bi-Level]                0.1548      0.130      1.195      0.232      -0.099       0.409
Style[T.Cape Cod]                0.2740      0.121      2.260      0.024       0.036       0.512
Style[T.Colonial]                0.3373      0.121      2.778      0.005       0.099       0.575
Style[T.Cottage]                -0.0194      0.122     -0.159      0.874      -0.259       0.220
Style[T.Dplx Bungalow]          -0.1922      0.101     -1.911      0.056      -0.389       0.005
Style[T.Duplex N/S]             -0.0573      0.101     -0.568      0.570      -0.255       0.140
Style[T.Duplex O/S]             -0.2727      0.100     -2.717      0.007      -0.469      -0.076
Style[T.Duplex-Cottage]         -0.4176      0.105     -3.994      0.000      -0.623      -0.213
Style[T.Mansion]                 0.3672      0.128      2.876      0.004       0.117       0.617
Style[T.Milwaukee Bungalow]      0.1290      0.121      1.064      0.288      -0.109       0.367
Style[T.Ranch]                   0.3379      0.121      2.786      0.005       0.100       0.576
Style[T.Residence O/S]           0.0998      0.121      0.824      0.410      -0.138       0.337
Style[T.Rm or Rooming House]    -0.2441      0.155     -1.577      0.115      -0.547       0.059
Style[T.Split Level]             0.1984      0.125      1.586      0.113      -0.047       0.444
Style[T.Townhouse]              -0.0811      0.104     -0.782      0.434      -0.284       0.122
Style[T.Triplex]                -0.2118      0.089     -2.392      0.017      -0.385      -0.038
Style[T.Tudor]                   0.3637      0.124      2.941      0.003       0.121       0.606
District[T.10]                   0.6678      0.015     45.862      0.000       0.639       0.696
District[T.11]                   0.7631      0.014     54.638      0.000       0.736       0.790
District[T.12]                   0.1575      0.023      6.765      0.000       0.112       0.203
District[T.13]                   0.7626      0.014     52.997      0.000       0.734       0.791
District[T.14]                   0.9436      0.015     64.681      0.000       0.915       0.972
District[T.15]                  -0.3656      0.021    -17.020      0.000      -0.408      -0.323
District[T.2]                    0.2498      0.016     15.489      0.000       0.218       0.281
District[T.3]                    1.1182      0.017     65.617      0.000       1.085       1.152
District[T.4]                   -0.1705      0.034     -5.076      0.000      -0.236      -0.105
District[T.5]                    0.6298      0.014     44.992      0.000       0.602       0.657
District[T.6]                   -0.0181      0.020     -0.905      0.365    

# Evaluacion de resultados

In [13]:
def MAE(prediction,true_values):
    return np.mean(                                                      # Mean
                np.abs(                                                   # Absolute
                        prediction-true_values                            # Error
                    )
                )

print("MAE entre model_log y log de Sale price:",MAE(model_log.predict(test) ,np.log(test.Sale_price)))
print("MAE entre exp(model_log) y true Sale price:",MAE(np.exp(model_log.predict(test)) ,test.Sale_price))

MAE entre model_log y log de Sale price: 0.2543504772499534
MAE entre exp(model_log) y true Sale price: 29693.10735966659


In [11]:
def RMSE(prediction,true_values):
    
    return np.sqrt(                                                          # Root
            np.mean(                                                      # Mean
                np.square(                                                # Squared
                         prediction-true_values                           # Error
                )
            )
        )
        
print("RMSE entre model_log y log de Sale price:", RMSE(model_log.predict(test) ,np.log(test.Sale_price)))
print("RMSE entre exp(model_log) y Sale price:",RMSE(np.exp(model_log.predict(test)) ,test.Sale_price))

RMSE between model_log and log of true Sale price: 0.3668708311367413
RMSE between exp(model_log) and true Sale price: 43035.68438059374


In [14]:
def MAPE(prediction,true_value):
    return np.mean(                                           # Mean
        np.abs(                                               # Absolute
               (prediction-true_value)/true_value             # Error
            )*100                                            # Percentage
    )

print("MAPE entre model_log y log de Sale price:", MAPE(model_log.predict(test) ,np.log(test.Sale_price)))
print("MAPE entre exp(model_log) y  Sale price:", MAPE(np.exp(model_log.predict(test)) ,test.Sale_price))

MAPE entre model_log y log de Sale price: 2.2448463666728182
MAPE entre exp(model_log) y  Sale price: 30.010358445800666
